### Set Environment

In [8]:
import json
import warnings
import pandas as pd
from glob2 import glob
warnings.filterwarnings("ignore")


### List all Output paths

In [9]:
jsonFiles = \
    glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json'
         )  # Can be used absolute or relative paths


### Function to
* __Merge all data in files__
* __Normalizes merged .json into dataframe__
* __Saves as csv__

In [ ]:
import re
import string
punc = string.punctuation.replace('@', '').replace(',', '')
pat = re.compile(f'[{punc}]')

In [31]:
tiktok_data = pd.DataFrame([])


def merge_JsonFiles(filename):
    for f1 in filename:
        global tiktok_data
        with open(f1, 'r') as infile:
            data = json.load(infile)
        tiktok_data = tiktok_data.append(pd.json_normalize(data),ignore_index=True)
        tiktok_data['mentions'] = tiktok_data['mentions'].replace(pat, '')
        tiktok_data.to_csv('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/tiktok.csv', index=False)


merge_JsonFiles(jsonFiles)


### Cleaning
#### Delete duplicate data: by 'webVideoUrl'

In [5]:
# Delete duplicate rows based on specific columns

tiktok_data = tiktok_data.drop_duplicates(subset=['webVideoUrl'],
                                          keep=False)


### Data Summaries
#### To Flourish:

### Visuals
#### 1. User from 'webVideoUrl'

In [6]:
source = pd.DataFrame([])
source["source"] = tiktok_data['webVideoUrl'].str.split('/').str[3]
source["source"].value_counts().rename_axis("unique_sources").reset_index(
    name="counts"
).to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/source.csv")


#### 2. Virial Video: 'webvideoUrl' 

In [7]:
tiktok_data["videoUrl"].value_counts(ascending=True).rename_axis(
    "unique_videos"
).reset_index(name="counts").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/videos.csv"
)


#### 3. Sentimental Analysis by 'text'

0        
1        
2        
3        
4        
       ..
3472     
3473     
3474     
3475     
3476     
Name: mentions, Length: 3477, dtype: object